<a href="https://colab.research.google.com/github/royorel/Lifespan_Age_Transformation_Synthesis/blob/master/LATS_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lifespan Age Transformation Synthesis Demo

This Colab notebook demonstrates the capabilities of the GAN architecture proposed in our paper. 

This colab lets you try our method on your own image!

First, let's download the github repository and install all dependencies.

In [ ]:
!git clone https://github.com/royorel/Lifespan_Age_Transformation_Synthesis
%cd Lifespan_Age_Transformation_Synthesis/
!pip3 install -r requirements.txt

Now let's download the pretrained models for males and females.

In [ ]:
!python download_models.py

Here, we import libraries and set options.

In [ ]:
import os
from collections import OrderedDict
from options.test_options import TestOptions
from data.data_loader import CreateDataLoader
from models.models import create_model
import util.util as util
from util.visualizer import Visualizer

opt = TestOptions().parse(save=False)
opt.display_id = 0 # do not launch visdom
opt.nThreads = 1   # test code only supports nThreads = 1
opt.batchSize = 1  # test code only supports batchSize = 1
opt.serial_batches = True  # no shuffle
opt.no_flip = True  # no flip
opt.in_the_wild = True # This triggers preprocessing of in the wild images in the dataloader
opt.traverse = True # This tells the model to traverse the latent space between anchor classes
opt.interp_step = 0.05 # this controls the number of images to interpolate between anchor classes

Don't worry about this message above, 
```
ipykernel_launcher.py: error: unrecognized arguments: -f /root/.local/share/jupyter/runtime/kernel-c9d47a98-bdba-4a5f-9f0a-e1437c7228b6.json
```
everything is perfectly fine...

Next on, we call the data loader and the visualizer class that generates the video from the network outputs.

In [ ]:
data_loader = CreateDataLoader(opt)
dataset = data_loader.load_data()
visualizer = Visualizer(opt)

Here, we define our model.

NOTE: if you plan to try the method for a female, change opt.name to 'females_model'.

In [ ]:
opt.name = 'males_model' # change to 'females_model' if you're trying the code on a female image
model = create_model(opt)
model.eval()

OK, it's time to upload your image.

For best results, use images according to the following guidelines:

1. The image should contain a single face.
2. Image was taken from a digital camera (phone cameras are fine). Old images from film cameras would produce low quality results.
3. Pure RGB images only. No black & white, grayscale, sepia, or filtered images (e.g. Instagram filters).
4. Person's head should directly face the camera. Looking sideways/downwards/upwards degrades the results.
5. The person's face should not be occluded (or partially occluded) by any item.
6. Both eyes should be open and visible. (eyeglasses are ok, no sunglasses)

Your uploaded images are local to the Colab instance and are not accessible by the paper authors

In [ ]:
# upload your image (the code supports only a single image at a time)
from google.colab import files
uploaded = files.upload()
for filename in uploaded.keys():
  img_path = filename
  print('User uploaded file "{name}"'.format(name=filename))

Finally, we preprocess the image, run the network, and save the result.

In [ ]:
data = dataset.dataset.get_item_from_path(img_path)
visuals = model.inference(data)

os.makedirs('results', exist_ok=True)
out_path = os.path.join('results', os.path.splitext(img_path)[0].replace(' ', '_') + '.mp4')
visualizer.make_video(visuals, out_path)

Let's display at the results.

NOTE: if you're using chrome, uncomment the lines below. For some reason, mp4 files won't display on chrome browser, so we need to convert to webm.

In [ ]:
use_webm = False
# For some unknown reason the mp4 video is not displayed on chrome
# If you have chrome, uncomment the following lines to convert the 
# result to webm for display purposes

# !pip3 install webm
# webm_out_path = os.path.join('results', os.path.splitext(img_path)[0].replace(' ', '_') + '.webm')
# !webm -i $out_path $webm_out_path
# use_webm = True

from IPython.display import HTML
from base64 import b64encode
video_path = webm_out_path if use_webm else out_path
video_type = "video/webm" if use_webm else "video/mp4"
mp4 = open(video_path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width={0} controls>
      <source src="{1}" type="{2}">
</video>
""".format(opt.fineSize, data_url, video_type))

You can download the result if you want

In [ ]:
files.download(out_path)